# Project1 - Team 5, Studio 1

Annie Chu, SeungU Lyu

### Question: How would the age demographics change over the course of 15 years if the US implemented an one-child policy?
Currently, the US population is 325.7 million, projected to be around 360 million by 2030. Data provided by the US Census also shows the greatest change in population among the 65+ age group, followed by the 18-44 age group, followed by the 45-64 group, and finally the under 18 group. This change may be explained by the evident in the declining fertility, which has dropped to an all time low of 1.76 children/female. 

We aimed to explore how the age group demographics would shift over 15 years if the US had implemented a one-child policy, essentially viewing how the US population would change if the fertility dropped to less than 1 child/female. 

In [8]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

# importing "copy" for copy operations 
import copy

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd

### Initial Data Pulls

#### Initial Population Data by Age Range and Gender (0-4, 5-9, etc)

In [10]:
pop_init_data = pd.read_csv('data/pop_2017_US.csv', index_col='age')

FileNotFoundError: File b'data/pop_2017_US.csv' does not exist

Source: https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?src=bkmk

#### Death Rate Data by Age Group and Gender (0, 1-4, 5-9, etc)

In [ ]:
death_rate_data = pd.read_csv('data/age_death_rate.csv', index_col='age')

Source: https://www.statista.com/statistics/241488/population-of-the-us-by-sex-and-age/

#### Probability of Woman Ever Having a Child by Age Group (15-19, 20-24, etc)

In [ ]:
child_ever_data = pd.read_csv('data/child_ever_born.csv', index_col='age')

Source: Fertility of Women in the United States: 2012, Lindsay M. Monte and Renee R. Ellis

#### Probability of Woman Having First Child by Age

In [ ]:
first_birth_data = pd.read_csv('data/first_birth_rate.csv', index_col='age')

Source: https://www.cdc.gov/nchs/nvss/births.htm

#### Adapting Initial Population Data into Age Groups

In [ ]:
male_pop = linspace(0,0,90)
female_pop = linspace(0,0,90)

ident = 4
for i in range(90):
    if i>ident:
        ident += 5
    male_pop[i] = int(pop_init_data.male[ident]/5)
    female_pop[i] = int(pop_init_data.female[ident]/5)

In [ ]:
female_pop[0] #test

#### Setting Initial Population State by Gender

In [ ]:
init_population = State(male = male_pop, female = female_pop)

#### Storing Model Parameters in a System Object

In [ ]:
system = System(male_death = death_rate_data.male,
               female_death = death_rate_data.female,
               child_ever = child_ever_data.percentage,
               first_rate = first_birth_data.percentage,
               t_0 = 2018,
               t_end = 2033,
               init_pop = init_population)

#### Creating State object to initialize arrays for the 3 different groups: Males, Females who haven't had the chance to have a child, Females who have had the chance to have a child

In [ ]:
population = State(male = copy.deepcopy(system.init_pop.male), female = copy.deepcopy(system.init_pop.female), female_w = linspace(0,0,90))

#### Assigning Population Value to 2 Female Groups Above

In [ ]:
ident = 19
for i in range(15,49):
    if i>ident:
        ident += 5
    population.female_w[i] = int(population.female[i]*system.child_ever[ident])
    population.female[i] = int(population.female[i]*(1-system.child_ever[ident]))

#### Implementation of Relative Gender Death Rate to 3 Groups

In [ ]:
def update_func_death(state,system):
    ident = 4
    state.male[0] = int(state.male[0] * (1-system.male_death[0]))
    state.female[0] = int(state.female[0] * (1-system.female_death[0]))
    for i in range(1,90):
        if i>ident:
            ident += 5
        state.male[i] = int(state.male[i] * (1-system.male_death[ident]))
        state.female[i] = int(state.female[i] * (1-system.female_death[ident]))
        state.female_w[i] = int(state.female_w[i] * (1-system.female_death[ident]))
    return state

#### Implementation of Birth Rate: Creating Newborns + Shifting Female Group (those who still have potential to give birth) to Female_W Group (those who no longer do)

In [ ]:
def update_func_birth(state,system):
    baby_total = 0
    for i in range(15,50):
        baby = int(state.female[i]*system.first_rate[i])
        state.female[i] -= baby
        state.female_w[i] += baby
        baby_total += baby
    return baby_total

#### Updating Population Age + New Births: Initializing New Births to Female/Male to 0 Age Group and Shifting All Ages Up by one

In [ ]:
def update_func_pop(baby,state,system):
    for i in range(89):
        k = 89-i
        state.male[k] = state.male[k-1]
        state.female[k] = state.female[k-1]
        state.female_w[k] = state.female_w[k-1]
    state.male[0] = int(baby/2)
    state.female[0] = int(baby/2)
    return state

#### General adding function to find total population among 3 groups (male, female, female_w)

In [ ]:
def addall(state):
    total = 0
    for i in range(90):
        total = total + state.male[i] + state.female[i] + state.female_w[i]
    return total

#### Function used to create TimeSeries with Total Population 

In [ ]:
def run_population(system, state, update_func_death, update_func_birth, update_func_pop, addall):
    nstate = State(male = copy.deepcopy(state.male), female = copy.deepcopy(state.female), female_w = copy.deepcopy(state.female_w))
    results = TimeSeries()
    for t in linrange(system.t_0, system.t_end):
        nstate = update_func_death(nstate,system)
        baby = update_func_birth(nstate,system)
        nstate = update_func_pop(baby,nstate,system)
        totalpop = addall(nstate)
        results[t+1] = totalpop
    return results

#### Adding function used to define what the age groups are and their subsequent total population within that age group
Shown later, we split the population into 5 age groups: 0-14, 15-30, 31-49, 50-70, 71-89. The end age is 89 because the initial data pull age limit is 89. 

In [ ]:
def agedemos(num_s, num_e, state):
    age_total = 0
    for i in range(num_s, num_e+1):
        age_total = age_total + state.male[i] + state.female[i] + state.female_w[i]
    return age_total

#### Function used to create TimeSeries with Total Age Group Values

In [ ]:
def age_group(system, state, update_func_death, update_func_birth, update_func_pop, agedemos):
    nstate = State(male = copy.deepcopy(state.male), female = copy.deepcopy(state.female), female_w = copy.deepcopy(state.female_w))
    demo_state = State(ag_one = TimeSeries(), ag_two = TimeSeries(), ag_three = TimeSeries(), ag_four = TimeSeries(), ag_five = TimeSeries())
    for t in linrange(system.t_0, system.t_end):
        nstate = update_func_death(nstate,system)
        baby = update_func_birth(nstate,system)
        nstate = update_func_pop(baby,nstate,system)
        demo_state.ag_one[t+1] = agedemos(0,14,nstate)
        demo_state.ag_two[t+1] = agedemos(15,30,nstate)
        demo_state.ag_three[t+1] = agedemos(31,49,nstate)
        demo_state.ag_four[t+1] = agedemos(50,70,nstate)
        demo_state.ag_five[t+1] = agedemos(71,89,nstate)
    return demo_state

#### Function used to create TimeSeries with Total Age Group Percentages (demographics)

In [ ]:
def age_group_per(system, state, update_func_death, update_func_birth, update_func_pop, agedemos, addall):
    nstate = State(male = copy.deepcopy(state.male), female = copy.deepcopy(state.female), female_w = copy.deepcopy(state.female_w))
    demo_state = State(ag_one = TimeSeries(), ag_two = TimeSeries(), ag_three = TimeSeries(), ag_four = TimeSeries(), ag_five = TimeSeries())
    for t in linrange(system.t_0, system.t_end):
        nstate = update_func_death(nstate,system)
        baby = update_func_birth(nstate,system)
        nstate = update_func_pop(baby,nstate,system)
        totalpop = addall(nstate)
        demo_state.ag_one[t+1] = agedemos(0,14,nstate)*100/totalpop
        demo_state.ag_two[t+1] = agedemos(15,30,nstate)*100/totalpop
        demo_state.ag_three[t+1] = agedemos(31,50,nstate)*100/totalpop
        demo_state.ag_four[t+1] = agedemos(51,70,nstate)*100/totalpop
        demo_state.ag_five[t+1] = agedemos(71,89,nstate)*100/totalpop
    return demo_state

Viewing Total Age Group Demographic (Values) by Year

In [ ]:
demo = age_group(system, population, update_func_death, update_func_birth, update_func_pop, agedemos)

Viewing Total Age Group Demographic (Percentages) by Year

In [ ]:
demo_per = age_group_per(system, population, update_func_death, update_func_birth, update_func_pop, agedemos, addall)

Viewing Total Population by Year

In [ ]:
results = run_population(system, population, update_func_death, update_func_birth, update_func_pop, addall)

Creating a TimeSeries Adding All Age Group Values to Check Consistency with Total Population

In [ ]:
check = TimeSeries()
for i in linrange(system.t_0, system.t_end):
    check = demo.ag_one + demo.ag_two + demo.ag_three + demo.ag_four + demo.ag_five
check


#### Plotting Results

Plotting Total Population by Year -- Line Graph

In [ ]:
plot(results, ':')

Plotting Age Group Demographics (Value) by Year -- Line Graph

In [ ]:
plot(demo.ag_one)
plot(demo.ag_two)
plot(demo.ag_three)
plot(demo.ag_four)
plot(demo.ag_five)
#plot(results)

Plotting Age Group Demographics (Value) by Year -- Stacked Bar Graph

In [13]:
# Values of each group
bars1 = demo.ag_one
bars2 = demo.ag_two
bars3 = demo.ag_three
bars4 = demo.ag_four
bars5 = demo.ag_five

# The position of the bars on the x-axis-timerange
r = linrange(system.t_0+1, system.t_end+1)

#setting bar width
barWidth = 0.97

plt.figure(figsize=(15, 8))
 
# Create brown bars
plt.bar(r, bars1, color='#7f6d5f', edgecolor='white', width=barWidth)
# Create green bars (middle), on top of the firs ones
plt.bar(r, bars2, bottom=bars1, color='#557f2d', edgecolor='white', width=barWidth)
# Create green bars (top)
plt.bar(r, bars3, bottom=bars1+bars2, color='#2d7f5e', edgecolor='white', width=barWidth)
# Create Blue bars
plt.bar(r, bars4, bottom=bars1+bars2+bars3, color='#2E9BC8', edgecolor='white', width=barWidth)
#Create yellow bars
plt.bar(r, bars5, bottom=bars1+bars2+bars3+bars4, color='#AFA928', edgecolor='white', width=barWidth)
 
# Custom axis
plt.xticks(r)
plt.xlabel("Time")
plt.ylabel("Population")
plt.title('U.S Population Over 15 years', fontweight = 'bold')
group = ['0-14','15-30','31-49','50-70','71-89']
plt.legend(group,loc=4)
 
# Show graphic
plt.show()


NameError: name 'demo' is not defined

Plotting Age Group Demographics (Percent) by Year -- Line Graph

In [ ]:
plot(demo_per.ag_one)
plot(demo_per.ag_two)
plot(demo_per.ag_three)
plot(demo_per.ag_four)
plot(demo_per.ag_five)

Plotting Age Group Demographics (Value) by Year -- Stacked Bar Graph

In [ ]:
# Values of each group
bars1 = demo_per.ag_one
bars2 = demo_per.ag_two
bars3 = demo_per.ag_three
bars4 = demo_per.ag_four
bars5 = demo_per.ag_five

# The position of the bars on the x-axis-timerange
r = linrange(system.t_0+1, system.t_end+1)

#setting bar width
barWidth = 0.97

plt.figure(figsize=(15, 8))
 
# Create brown bars
plt.bar(r, bars1, color='#7f6d5f', edgecolor='white', width=barWidth)
# Create green bars (middle), on top of the firs ones
plt.bar(r, bars2, bottom=bars1, color='#557f2d', edgecolor='white', width=barWidth)
# Create green bars (top)
plt.bar(r, bars3, bottom=bars1+bars2, color='#2d7f5e', edgecolor='white', width=barWidth)
# Create Blue bars
plt.bar(r, bars4, bottom=bars1+bars2+bars3, color='#2E9BC8', edgecolor='white', width=barWidth)
#Create yellow bars
plt.bar(r, bars5, bottom=bars1+bars2+bars3+bars4, color='#AFA928', edgecolor='white', width=barWidth)
 
# Custom axis
plt.xticks(r)
plt.xlabel("Time")
plt.ylabel("Percentage")
plt.title('U.S Age Demographic over 15 years', fontweight = 'bold')

group = ['0-14','15-30','31-49','50-70','71-89']
plt.legend(group,loc=4)
 
# Show graphic
plt.show()

The graphs above show the change in age demographics within the US over the course of 15 years if a one-child policy was implemented. 

Based on this model, total population seems to be dropping at a fairly linear rate. Within the total population, the 

This model 



Limitations:

1. Not accounting for twins
2. birth/death rate assuming constant for next 15 years
3. Assumption of gender birth ratio
4. No immigrants
5. Data not the most recent 
